Charles Vin & Aymeric Delefosse <span style="float:right">DAC</span>

# TME 7 - Approches par modèles de substitution


### Utilities


In [1]:
import pandas as pd
import os
from collections import Counter
from typing import Any
import numpy as np
from copy import deepcopy
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from icecream import ic
from tqdm import tqdm
from distances import simple_match_distance

### Classes

**Différence par rapport au TME précédent** : Création d'une classe `Criterion` qui permet de changer la mesure de discrimination utilisée pour construire l'arbre. Les différents critères sont l'entropie de Shannon, l'indice de diversité de Gini et la mesure d'ambiguïté de Yuan & Shaw. Par défaut, l'arbre utilise l'indice de Gini.


In [2]:
class Criterion:
    def __init__(self, criterion) -> None:
        self.criterion = criterion

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        if self.criterion == "entropy":
            return self.shannon(*args, **kwds)
        elif self.criterion == "gini":
            return self.gini(*args, **kwds)
        elif self.criterion == "ambiguity":
            return self.ambiguity(*args, **kwds)
        else:
            raise ValueError(f"Unknown impurity criterion: {self.criterion}")

    @staticmethod
    def shannon(labels):
        """Permet de calculer l'entropie de Shannon.

        Renvoie : $\sum_{k=1}^{K} p(c_k|v_j)\log p(c_k|v_j)$.

        \[
            H_S(Y|X) = -\sum{j=1}^{m} p(v_j) \cdot \sum_{k=1}^{K} p(c_k|v_j)\log p(c_k|v_j)
        \]
        """
        label_counts = Counter(labels)
        probabilities = [count / len(labels) for count in label_counts.values()]
        return -sum(p * np.log2(p) for p in probabilities if p > 0)

    @staticmethod
    def gini(labels):
        """Permet de calculer l'indice de diversité de Gini.

        Renvoie : $(1 - \sum_{k=1}^{K} p(c_k | v_j)^2)$.

        \[
            H_G(Y | X) = \sum_{j=1}^{m} p(v_j) \cdot (1 - \sum_{k=1}^{K} p(c_k | v_j)^2)
        \]
        """
        label_counts = Counter(labels)
        probabilities = [count / len(labels) for count in label_counts.values()]
        return 1 - sum(p**2 for p in probabilities)

    @staticmethod
    def ambiguity(labels):
        """Permet de calculer la mesure d'ambiguïté [Yuan & Shaw, 1995].

        Renvoie la mesure de non-spécificité.

        \[
            H_Y(Y|X) = \sum_{j=1}^{m} p(v_j) \cdot g(\Pi(Y|v_j))
        \]

        avec $g$ une mesure de non-spécificité :

        \[
            g(\Pi(C|v_j)) = \sum_{i=2}^{K} \pi_i \cdot (\log(i) - \log(i-1))
        \]

        où $\pi$ est obtenue de la façon suivante :
        1. on ordonne les $p(c_k|v_j) dans l'odre décroissant ;
        2. on définit $\pi_i = \frac{p_i}{p_1}$ pour tout $i = 1, ..., K$.
        """
        label_counts = Counter(labels)
        probabilities = [count / len(labels) for count in label_counts.values()]

        # Calculate π
        probabilities.sort(reverse=True)
        pi = [prob / probabilities[0] for prob in probabilities]

        # Calculate the measure of non-specificity g(Π(C|v_j))
        g = sum(pi[i] * (np.log2(i + 1) - np.log2(i)) for i in range(1, len(pi) + 1))

        return g

**Différences par rapport au TME précédent** :

- Initialisation d'une mesure de discrimination différente ;
- `find_counterfactuals` : trouve les règles contrefactuelles en parcourant l'arbre dans l'autre sens grâce à `untrace_tree` ;
- `find_and_count` : renvoie **toutes** les règles contre-factuelles grâce à `find_couterfactuals`, la classe prédite et le nombre de tests invalidés pour un exemple donné grâce à `count_rules` ;
- `explain_counterfactuals` : renvoie les **meilleures** règles contre-factuelles, c.-à-d. les règles $r_{best}$ qui minimisent $n_r'$, où $n_r'$ est le nombre de tests de $r'$ invalidés par $\mathbf{x}$ ;
- `explain_notebook` : présente l'explication fournie à l'utilisateur d'une manière claire, où les règles invalidés sont mises en couleurs ;


In [3]:
class SymbolicDecisionTree:
    """Un arbre de décision prenant en compte des données symboliques."""

    def __init__(self, criterion="gini", feature_values=None):
        self.tree = None
        self.criterion = Criterion(criterion)
        self.feature_values = feature_values or {}

    class Node:
        """Un noeud de l'arbre de décision symbolique."""

        def __init__(
            self,
            feature=None,
            value=None,
            true_branch=None,
            false_branch=None,
            result=None,
            entropy=None,
        ):
            self.feature = feature
            self.value = value
            self.true_branch = true_branch
            self.false_branch = false_branch
            self.result = result
            self.entropy = entropy

    def fit(self, X, y):
        """Construit l'arbre de décision à partir de l'ensemble d'apprentissage (X, y)."""
        training_data = pd.concat([X, y], axis=1)
        self.tree = self.build_tree(training_data)

    def generate_alternative_rules(self, feature, value):
        """Génération des règles alternatives pour une caractéristique basées sur ses
        valeurs possibles.
        """
        return [
            (feature, True, alt_value)
            for alt_value in self.feature_values[feature]
            if alt_value != value
        ]

    def predict(self, X):
        """Prédit les classes pour des échantillons d'entrée."""
        if isinstance(X, pd.DataFrame):
            predictions = [
                self.predict_single_entry(entry) for _, entry in X.iterrows()
            ]
        elif isinstance(X, pd.Series):
            predictions = self.predict_single_entry(X)
        else:
            raise TypeError(
                f"type {type(X)} not supported in predict, please implement or convert to a pandas object"
            )
        return predictions

    def predict_single_entry(self, entry):
        """Prédit la classe pour une seule entrée."""
        node = self.tree
        while node.result is None:
            if entry[node.feature] == node.value:
                node = node.true_branch
            else:
                node = node.false_branch
        return node.result

    def predict_xai(self, entry):
        """Fournit une explication pour un exemple donné."""
        explanation, _ = self.trace_tree(entry, self.tree, [])
        return explanation

    def find_counterfactuals(self, class_to_exclude):
        """Trouve les règles contre-factuelles de l'arbre pour un exemple donné.

        :param class_to_exclude: La classe de l'exemple à interpréter.
        """
        paths = self.untrace_tree(self.tree, [])
        filtered_paths = [path for path in paths if path[-1] != class_to_exclude]
        return filtered_paths

    def find_and_count(self, entry, class_to_exclude):
        """Renvoie **toutes** les règles contre-factuelles, la classe prédite et le
        nombre de tests invalidés pour un exemple donné.

        :param entry: L'exemple à interpréter.
        :param class_to_exclude: La classe de l'exemple à interpréter.
        """
        formatted_path = []
        counterfactuals = self.find_counterfactuals(class_to_exclude)
        unsatisfied_rules = self.count_rules(entry, counterfactuals)
        for path, rule_count in zip(counterfactuals, unsatisfied_rules):
            # formatted_path.append(self.format_path(path[0], path[1]))
            formatted_path.append(
                (
                    [
                        f"{feature} {'!=' if not decision else '=='} {value}"
                        for feature, decision, value in path[0]
                    ],
                    path[1],
                    rule_count,
                )
            )
        return formatted_path

    def explain_counterfactuals(self, entry, class_to_exclude):
        """Renvoie les **meilleures** règles contre-factuelles, la classe prédite et le
        nombre de tests invalidés pour un exemple donné.

        :param entry: L'exemple à interpréter.
        :param class_to_exclude: La classe de l'exemple à interpréter.
        """
        formatted_path = []
        counterfactuals = self.find_counterfactuals(class_to_exclude)
        unsatisfied_rules = self.count_rules(entry, counterfactuals)

        # Trouve les règles qui minimisent le nombre de tests invalidés par l'exemple
        min_count = min(unsatisfied_rules)
        best_indices = [
            i for i, count in enumerate(unsatisfied_rules) if count == min_count
        ]
        best_rules = [counterfactuals[i] for i in best_indices]

        for path, rule_count in zip(best_rules, unsatisfied_rules):
            # formatted_path.append(self.format_path(path[0], path[1]))
            formatted_path.append(
                (
                    [
                        f"{feature} {'!=' if not decision else '=='} {value}"
                        for feature, decision, value in path[0]
                    ],
                    path[1],
                    rule_count,
                )
            )
        return formatted_path

    def explain_notebook(self, entry, class_to_exclude):
        """Présente l'explication fournie à l'utilisateur d'une manière claire, où les
        règles invalidés sont mises en couleurs.

        :param entry: L'exemple à interpréter.
        :param class_to_exclude: La classe de l'exemple à interpréter.
        """
        desc = ", ".join(
            [
                f"{feature} = {value}"
                for feature, value in zip(entry.index, entry.values)
            ]
        )
        explanation = (
            f"<p><strong>Description</strong> : {desc}, Classe = {class_to_exclude}</p>"
        )
        # Régle de classification
        prediction = self.predict(entry)
        explanation += f"<p><strong>Règle de classification</strong> : l'exemple est classé {prediction}<br/>"
        classification_rule = self.predict_xai(entry)
        classification_rule = [path.split(" ") for path in classification_rule]
        explanation += self.format_path(classification_rule, class_to_exclude, entry)
        explanation += f"</p><p><strong>Règles contre-exemples</strong> : celles dont la conclusion n'est pas {prediction}<br/>"

        # Règles contre-exemples
        counterfactuals = self.explain_counterfactuals(entry, class_to_exclude)
        for cf in counterfactuals:
            rules = [rule.split(" ") for rule in cf[0]]
            explanation += self.format_path(rules, cf[1], entry)
            explanation += "<br/>"
        explanation += "</p>"

        return display(HTML(explanation))

    @staticmethod
    def format_path(path, result, entry):
        """Formatte un chemin en un human-readable string représentant les règles de
        décision. Les règles qui diffèrent des valeurs d'entrée sont mises en évidence
        en couleur.
        """
        conditions = []
        for feature, decision, value in path:
            verbe = "est" if decision == "==" else "n'est pas"
            if entry[feature] != value and verbe == "est":
                condition = (
                    f"<span style='color: #648FFF;'>{feature} {verbe} {value}</span>"
                )
            else:
                condition = f"{feature} {verbe} {value}"
            conditions.append(condition)

        if len(conditions) > 1:
            conditions_str = ", ".join(conditions[:-1]) + " et " + conditions[-1]
        else:
            conditions_str = conditions[0]

        return f"Si {conditions_str} alors classe {result}."

    def build_tree(self, data):
        # If there's no data, or if all targets are the same, return a leaf node with
        # the result
        if len(data) == 0:
            return self.Node()

        current_uncertainty = self.criterion(data.iloc[:, -1])
        best_gain = 0
        best_criteria = None
        best_sets = None

        feature_count = len(data.columns) - 1  # number of attributes

        for col in range(feature_count):  # for each feature
            feature_values = data.iloc[:, col].unique()  # unique values
            for val in feature_values:  # for each value
                partitioned_data = self.partition(data, data.columns[col], val)

                # Information gain
                p = float(partitioned_data[0].shape[0]) / data.shape[0]
                gain = (
                    current_uncertainty
                    - p * self.criterion(partitioned_data[0].iloc[:, -1])
                    - (1 - p) * self.criterion(partitioned_data[1].iloc[:, -1])
                )

                if (
                    gain > best_gain
                    and len(partitioned_data[0]) > 0
                    and len(partitioned_data[1]) > 0
                ):
                    best_gain = gain
                    best_criteria = (data.columns[col], val)
                    best_sets = partitioned_data

        if best_gain > 0:
            true_branch = self.build_tree(best_sets[0])
            false_branch = self.build_tree(best_sets[1])
            return self.Node(
                feature=best_criteria[0],
                value=best_criteria[1],
                true_branch=true_branch,
                false_branch=false_branch,
                entropy=current_uncertainty,
            )
        else:
            # We're at a leaf, determine the outcome most frequent class
            outcome = data.iloc[:, -1].value_counts().idxmax()
            return self.Node(result=outcome, entropy=current_uncertainty)

    @staticmethod
    def partition(data, feature, value):
        true_data = data[data[feature] == value]
        false_data = data[data[feature] != value]
        return true_data, false_data

    def trace_tree(self, entry, node, explanation=[]):
        """Collecte le chemin mené par l'arbre pour prédire un exemple donné.

        :param entry: L'exemple à interpréter.
        :param node: Le noeud actuel de l'arbre.
        :param explanation: Le chemin mené par l'arbre.
        """
        if node.result is not None:
            return explanation, node.result

        if entry[node.feature] == node.value:
            explanation.append(f"{node.feature} == {node.value}")
            return self.trace_tree(entry, node.true_branch, explanation)
        else:
            explanation.append(f"{node.feature} != {node.value}")
            return self.trace_tree(entry, node.false_branch, explanation)

    def untrace_tree(self, node, path=[], entry={}):
        """Traverse récursivement l'arbre de décision pour collecter les chemins menant
        aux noeuds feuilles.

        :param node: Le noeud actuel de l'arbre.
        :param path: Le chemin emprunté pour atteindre le nœud courant.
        :return: Une liste de chemins, où chaque chemin est une liste de tuples
        contenant (feature, decision, value) menant à une feuille.
        """
        if node.result is not None:
            # Return the path with the result at the end
            return [(path, node.result)]

        paths = []
        if node.false_branch is not None:
            # Traverse the false branch
            # Generate alternative rules for the false branch
            alternative_rules = self.generate_alternative_rules(
                node.feature, node.value
            )
            for rule in alternative_rules:
                alt_path = path[:] + [rule]
                paths.extend(self.untrace_tree(node.false_branch, alt_path, entry))

        if node.true_branch is not None:
            # Traverse the true branch
            true_path = path[:] + [(node.feature, True, node.value)]
            paths.extend(self.untrace_tree(node.true_branch, true_path, entry))

        return paths

    def count_rules(self, entry, counterfactuals):
        """Compte le nombre de tests invalidés des règles contre-factuelles pour un
        échantillon donné.

        :param entry: L'exemple à interpréter.
        :param counterfactuals: L'ensemble les règles contre-factuelles.
        """
        unsatisfied_rules_count = []
        for path in counterfactuals:
            count = sum(
                1
                for feature, decision, value in path[0]
                if (decision is False and entry[feature] == value)
                or (decision is True and entry[feature] != value)
            )
            unsatisfied_rules_count.append(count)
        return unsatisfied_rules_count

    def display_tree(self, node=None, indent="", branch=""):
        """Visualise la structure arborescente de l'arbre dans un format clair et
        organisé.

        Format de sortie :
        "├──" indique un noeud qui confirme la condition.
        "└──" indique un noeud qui ne confirme pas la condition.
        Les feuilles affichent le résultat de la classe.
        Les noeuds internes affichent le critère de décision et son entropie.

        Exemple :
        feature_name1 == feature_value1? (Entropy = 0.1234)
        ├── Class : Class_A
        └── feature_name2 == feature_value2? (Entropy = 0.5678)
            ├── Class : Class_B
            └── Class : Class_C

        Dans cet exemple :
        - Si feature_name1 == feature_value1, l'arbre de décision le classe comme
        'Class_A'.
        - Sinon, il vérifie si feature_name2 == feature_value2 ; si c'est vrai, il est
        classé comme 'Class_B', et si c'est faux, il est classé comme 'Class_C'.
        """
        if node is None:
            node = self.tree

        # Base case: if it's a leaf node, print the result and return
        if node.result is not None:
            print(f"{indent}{branch}Class: {node.result}")
            return

        # Print the criterion for the current node
        print(
            f"{indent}{branch}{node.feature} == {node.value}? (Entropy = {node.entropy:.4f})"
        )

        # Recursive case: print the true and false branches
        new_indent = indent + ("│   " if branch == "├── " else "    ")

        # For true branch, use '├──'
        self.display_tree(node.true_branch, new_indent, "├── ")

        # For false branch, use '└──'
        self.display_tree(node.false_branch, new_indent, "└── ")

## Construction de règles contre-factuelles


Séparer l'ensemble de données (`predictions`) en une base d’apprentissage et une base de test :


In [4]:
PATH = "."
elections_data_path = f"{PATH}/data/elections.csv"

# Séparer les caractéristiques et les étiquettes
elections_data = pd.read_csv(elections_data_path)
X = elections_data.drop("Label", axis=1)
y = elections_data["Label"]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
elections_data

,Adresse,Majeur?,Nationalite,Label
0,Paris,oui,Francais,1
1,Paris,non,Francais,-1
2,Montpellier,oui,Italien,1
3,Paris,oui,Suisse,-1
4,Strasbourg,non,Italien,-1
5,Strasbourg,non,Francais,-1
6,Strasbourg,oui,Francais,1
7,Montpellier,oui,Suisse,-1


In [5]:
feature_values = {col: X[col].unique().tolist() for col in X.columns}
feature_values

{'Adresse': ['Paris', 'Montpellier', 'Strasbourg'],
 'Majeur?': ['oui', 'non'],
 'Nationalite': ['Francais', 'Italien', 'Suisse']}

Construction d'un arbre de décision avec la base d'apprentissage :


In [6]:
# Initialiser et entraîner l'arbre de décision symbolique
tree_xai = SymbolicDecisionTree(feature_values=feature_values)
tree_xai.fit(X_train, y_train)

# Évaluer la précision de l'arbre de décision
predictions = tree_xai.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.4f}%")

Accuracy: 100.0000%


In [7]:
tree_xai.display_tree()

Nationalite == Suisse? (Entropy = 0.4800)
    ├── Class: -1
    └── Majeur? == oui? (Entropy = 0.4444)
        ├── Class: 1
        └── Class: -1


Application de l'arbre sur un exemple $\mathbf{x}$ et affichage de la règle activée :


In [8]:
sample, sample_label = X.iloc[0], y.iloc[0]
majeur_str = (
    "est une personne majeure"
    if sample["Majeur?"]
    else "n'est pas une personne majeure"
)
vote_str = "peut voter" if sample_label else "ne peut pas voter"
predict_str = "votant" if tree_xai.predict(sample) == 1 else "non votant"
print(
    f"L'exemple qu'on cherche à expliquer {majeur_str} de nationalité {sample['Nationalite']}."
)
print(f"Il {vote_str}.")
print(f"Il est classé comme de la classe {predict_str}.")
print(f"Il active les règles suivante de l'arbre {tree_xai.predict_xai(sample)}.")

L'exemple qu'on cherche à expliquer est une personne majeure de nationalité Francais.
Il peut voter.
Il est classé comme de la classe votant.
Il active les règles suivante de l'arbre ['Nationalite != Suisse', 'Majeur? == oui'].


Affichage de toutes les règles contre-factuelles, c.-à-d. les règles non activés $r'$, la classe prédite par ces règles et le nombre de tests invalidés $ n\_{r'}$ par $\mathbf{x}$ :


In [9]:
tree_xai.find_and_count(sample, sample_label)

[(['Nationalite == Francais', 'Majeur? == non'], -1, 1),
 (['Nationalite == Italien', 'Majeur? == non'], -1, 2),
 (['Nationalite == Suisse'], -1, 1)]

Explication de $\mathbf{x}$, c.-à-d. les règles qui minimisent le nombre de tests invalidés :


In [10]:
tree_xai.explain_counterfactuals(sample, sample_label)

[(['Nationalite == Francais', 'Majeur? == non'], -1, 1),
 (['Nationalite == Suisse'], -1, 2)]

Présentation à l'utilisateur :


In [11]:
tree_xai.explain_notebook(sample, sample_label)

## Génération de bases d’apprentissage et d’explications


### Base d'apprentissage - Méthode LORE

Algorithme génétique


In [12]:
class GeneticAlgorithm:
    def __init__(
        self,
        distance,
        pc=0.5,
        pm=0.5,
        nb_genetics=100,
        N=1000,
    ):
        self.N = N
        self.pc = pc
        self.pm = pm
        self.nb_genetics = nb_genetics
        self.distance = distance

        self.model = None
        self.value_distribution = None

    def fit(self, test_set, model):
        """Adapte l'algorithme au modèle donné et au jeu de données de test.

        :param test_set: Jeu de données de test.
        :param model: Modèle de machine learning à utiliser.
        """
        self.model = model
        self.value_distribution = {}
        for col in test_set.columns:
            vc = test_set[col].value_counts(normalize=True)
            self.value_distribution[col] = (vc.index, vc.values)

    def predict(self, sample):
        """Prédit les labels pour un échantillon donné en utilisant le modèle et
        l'algorithme génétique.

        :param sample: Échantillon pour lequel effectuer la prédiction.
        :return: Un tuple contenant les échantillons générés et leurs labels prédits.
        """
        Z_true = self.geneticNeigh(sample, True)
        Z_false = self.geneticNeigh(sample, False)
        Z = pd.concat((Z_true, Z_false)).reset_index(drop=True)
        Z_hat = pd.Series(self.model.predict(Z), name="Label")
        return Z, Z_hat

    def geneticNeigh(self, x: pd.Series, same: bool):
        """Génère un voisinage génétique pour un échantillon donné.

        :param x: Échantillon original.
        :param same: Booléen déterminant si le voisinage doit avoir le même label que
        l'échantillon original.
        :return: DataFrame représentant le voisinage génétique.
        """
        if same:
            fitness = self.fitness_same
        else:
            fitness = self.fitness_dif

        P = pd.DataFrame([x] * self.N)  # Population init
        M = self.evaluate(x, P, fitness)
        top_M = np.flip(M.argsort())
        for _ in tqdm(range(self.nb_genetics)):
            P = P.iloc[top_M].head(self.N)
            P = self.crossover(P)
            P = self.mutate(P)
            M = self.evaluate(x, P, fitness)
            top_M = np.flip(M.argsort())
        return P

    def evaluate(self, x, P: pd.DataFrame, fitness):
        """Évalue la fitness de chaque individu dans la population.

        :param x: Échantillon original.
        :param P: Population à évaluer.
        :param fitness: Fonction de fitness à utiliser.
        :return: Liste des scores de fitness pour la population.
        """
        # x_hat = self.model.predict(x)
        # M_list = []
        # for _, z_i in P.iterrows():
        #     M_list.append(fitness(x, x_hat, z_i))
        # return np.array(M_list)
        x_hat = self.model.predict(x)
        return np.array([fitness(x, x_hat, z_i) for _, z_i in P.iterrows()])

    def fitness_same(self, x, x_hat, z_i):
        """Calcule la fitness pour un individu, en considérant que l'individu doit avoir
        le même label que l'échantillon original.

        :param x: Échantillon original.
        :param x_hat: Label prédit de l'échantillon original.
        :param z_i: Individu de la population pour lequel la fitness est calculée.
        :return: Score de fitness pour l'individu.
        """
        z_i_hat = self.model.predict(z_i)
        return (z_i_hat == x_hat) + (1 - self.distance(x, z_i)) - ((x == z_i).all())

    def fitness_dif(self, x, x_hat, z_i):
        """Calcule la fitness pour un individu, en considérant que l'individu doit avoir
        un label différent de l'échantillon original.

        :param x: Échantillon original.
        :param x_hat: Label prédit de l'échantillon original.
        :param z_i: Individu de la population pour lequel la fitness est calculée.
        :return: Score de fitness pour l'individu.
        """
        z_i_hat = self.model.predict(z_i)
        return (z_i_hat != x_hat) + (1 - self.distance(x, z_i)) - ((x == z_i).all())

    def mutate(self, P: pd.DataFrame):
        """Effectue des mutations sur une fraction de la population.

        :param P: Population actuelle.
        :return: Nouvelle population après mutation.
        """
        idx_to_mutate = np.random.randint(0, len(P), size=int(len(P) * self.pm))
        mutations = P.iloc[idx_to_mutate].apply(self.mutate_one, axis=1)
        P_mutated = pd.concat((mutations, P))
        return P_mutated

    def mutate_one(self, parent):
        """Effectue une mutation sur un seul individu.

        :param parent: Individu (parent) à muter.
        :return: Nouvel individu (enfant) après mutation.
        """
        attribute_list = parent.index
        att_1 = np.random.choice(attribute_list)
        att_2 = np.random.choice(attribute_list.drop(att_1))
        children = parent.copy()
        children[att_1] = np.random.choice(
            self.value_distribution[att_1][0], p=self.value_distribution[att_1][1]
        )
        children[att_2] = np.random.choice(
            self.value_distribution[att_2][0], p=self.value_distribution[att_2][1]
        )
        return children

    def crossover(self, P: pd.DataFrame):
        """Effectue un crossover sur une fraction de la population.

        :param P: Population actuelle.
        :return: Nouvelle population après crossover.
        """
        nb_parents = int(len(P) * self.pm)
        idx_parents = np.random.randint(0, len(P), size=(2, nb_parents // 2))
        crossovers = pd.concat(
            (
                self.crossover_one(P.iloc[idx_parents[0, i]], P.iloc[idx_parents[1, i]])
                for i in range(nb_parents // 2)
            )
        )
        P_mutated = pd.concat((crossovers, P))
        return P_mutated

    @staticmethod
    def crossover_one(parent_1, parent_2):
        """Effectue un crossover entre deux individus.

        :param parent_1: Premier parent pour le crossover.
        :param parent_2: Deuxième parent pour le crossover.
        :return: Deux nouveaux individus (enfants) après crossover.
        """
        attribute_list = parent_1.index
        # On veut deux attributs différents
        att_1 = np.random.choice(attribute_list)
        att_2 = np.random.choice(attribute_list.drop(att_1))
        att_list = [att_1, att_2]

        children_1 = parent_1.copy()
        children_1[att_list] = parent_2[att_list]

        children_2 = parent_2.copy()
        children_2[att_list] = parent_1[att_list]

        return pd.DataFrame((children_1, children_2))

In [13]:
sample = X.iloc[0]
geneticAlgorithm = GeneticAlgorithm(simple_match_distance)
geneticAlgorithm.fit(X, tree_xai)
Z, Z_hat = geneticAlgorithm.predict(sample)
pd.concat((Z, Z_hat), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:27<00:00,  1.14it/s]


,Adresse,Majeur?,Nationalite,Label
0,Montpellier,oui,Italien,1
1,Strasbourg,non,Francais,-1
2,Paris,oui,Francais,1
3,Paris,non,Francais,-1
4,Strasbourg,oui,Italien,1
...,...,...,...,...
4495,Paris,non,Francais,-1
4496,Paris,non,Francais,-1
4497,Paris,non,Francais,-1
4498,Paris,non,Francais,-1


In [14]:
X_train

,Adresse,Majeur?,Nationalite
7,Montpellier,oui,Suisse
2,Montpellier,oui,Italien
4,Strasbourg,non,Italien
3,Paris,oui,Suisse
6,Strasbourg,oui,Francais


## Explications


Contruction de l'arbre :


In [15]:
substitution_tree = SymbolicDecisionTree(feature_values=feature_values)
substitution_tree.fit(Z, Z_hat)
substitution_tree.display_tree()

Majeur? == oui? (Entropy = 0.5000)
    ├── Nationalite == Suisse? (Entropy = 0.1831)
    │   ├── Class: -1
    │   └── Class: 1
    └── Class: -1


Application de l'arbre sur un exemple $\mathbf{x}$ :


In [16]:
sample, sample_label = X.iloc[0], y.iloc[0]
majeur_str = (
    "est une personne majeure"
    if sample["Majeur?"]
    else "n'est pas une personne majeure"
)
vote_str = "peut voter" if sample_label else "ne peut pas voter"
predict_str = "votant" if tree_xai.predict(sample) == 1 else "non votant"
print(
    f"L'exemple qu'on cherche à expliquer {majeur_str} de nationalité {sample['Nationalite']}."
)
print(f"Il {vote_str}.")
print(f"Il est classé comme de la classe {predict_str}.")
print(f"Il active les règles suivante de l'arbre {tree_xai.predict_xai(sample)}.")

L'exemple qu'on cherche à expliquer est une personne majeure de nationalité Francais.
Il peut voter.
Il est classé comme de la classe votant.
Il active les règles suivante de l'arbre ['Nationalite != Suisse', 'Majeur? == oui'].


Explication de $\mathbf{x}$ :


In [17]:
substitution_tree.explain_counterfactuals(sample, sample_label)

[(['Majeur? == non'], -1, 1),
 (['Majeur? == oui', 'Nationalite == Suisse'], -1, 1)]

In [18]:
tree_xai.explain_notebook(sample, sample_label)

## Expérimentations


### Mushrooms


In [19]:
# Vérifier les fichiers extraits et trouver les fichiers de données "Mushrooms"
mushroom_data_path = f"{PATH}/data/"
extracted_files = os.listdir(mushroom_data_path)
mushroom_files = [file for file in extracted_files if file.startswith("mushroom")]

# Charger les données de chaque fichier mushroom et les combiner en un seul DataFrame
mushroom_dataframes = [
    pd.read_csv(os.path.join(mushroom_data_path, file)) for file in mushroom_files
]
combined_mushroom_data = pd.concat(mushroom_dataframes, ignore_index=True)

# Afficher les premières lignes des données combinées
display(combined_mushroom_data.columns)

# Séparer les caractéristiques et les étiquettes
X_mushrooms = combined_mushroom_data.drop(columns=["class"])  # features
y_mushrooms = combined_mushroom_data["class"]  # labels
feature_mushrooms = {
    col: X_mushrooms[col].unique().tolist() for col in X_mushrooms.columns
}

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X_mushrooms, y_mushrooms, test_size=0.4, random_state=42
)

# Initialiser et entraîner l'arbre de décision symbolique
tree_mushroom = SymbolicDecisionTree(feature_values=feature_mushrooms)
tree_mushroom.fit(X_train, y_train)

y_pred_mushrooms = tree_mushroom.predict(X_test)

# Évaluer la précision de l'arbre de décision
accuracy_mushrooms = accuracy_score(y_test, y_pred_mushrooms)
print(f"Accuracy: {accuracy_mushrooms * 100:.4f}%")

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

Accuracy: 100.0000%


In [20]:
sample_mushroom, sample_label_mushroom = X_mushrooms.iloc[0], y_mushrooms.iloc[0]
geneticAlgorithm_mushroom = GeneticAlgorithm(simple_match_distance)
geneticAlgorithm_mushroom.fit(X_mushrooms, tree_mushroom)
Z_mushroom, Z_hat_mushroom = geneticAlgorithm_mushroom.predict(sample_mushroom)

100%|██████████| 100/100 [01:41<00:00,  1.02s/it]


In [21]:
substitution_tree_mushroom = SymbolicDecisionTree(feature_values=feature_mushrooms)
substitution_tree_mushroom.fit(Z_mushroom, Z_hat_mushroom)
substitution_tree_mushroom.display_tree()

odor == f? (Entropy = 0.5000)
    ├── stalk-root == r? (Entropy = 0.0018)
    │   ├── Class: e
    │   └── Class: p
    └── odor == p? (Entropy = 0.0327)
        ├── Class: p
        └── odor == c? (Entropy = 0.0234)
            ├── Class: p
            └── odor == y? (Entropy = 0.0157)
                ├── Class: p
                └── odor == s? (Entropy = 0.0088)
                    ├── Class: p
                    └── stalk-root == c? (Entropy = 0.0027)
                        ├── Class: p
                        └── odor == m? (Entropy = 0.0009)
                            ├── Class: p
                            └── Class: e


In [22]:
substitution_tree_mushroom.explain_counterfactuals(
    sample_mushroom, sample_label_mushroom
)

[(['odor == n',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  1),
 (['odor == a',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == l',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == y',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == s',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == c',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == p',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == f',
   'stalk-root == b',
   'odor == f'],
  'e',
  2),
 (['odor == m',
   'odor == f',
   'odor == f',
   'odor == f',
   'odor == 

In [23]:
substitution_tree_mushroom.explain_notebook(sample_mushroom, sample_label_mushroom)

In [24]:
sample_mushroom, sample_label_mushroom = X_mushrooms.iloc[42], y_mushrooms.iloc[42]
geneticAlgorithm_mushroom = GeneticAlgorithm(simple_match_distance)
geneticAlgorithm_mushroom.fit(X_mushrooms, tree_mushroom)
Z_mushroom, Z_hat_mushroom = geneticAlgorithm_mushroom.predict(sample_mushroom)

100%|██████████| 100/100 [03:05<00:00,  1.86s/it]


In [25]:
substitution_tree_mushroom = SymbolicDecisionTree(feature_values=feature_mushrooms)
substitution_tree_mushroom.fit(Z_mushroom, Z_hat_mushroom)
substitution_tree_mushroom.display_tree()

odor == n? (Entropy = 0.5000)
    ├── gill-size == b? (Entropy = 0.0203)
    │   ├── Class: e
    │   └── Class: p
    └── odor == l? (Entropy = 0.0210)
        ├── Class: e
        └── odor == a? (Entropy = 0.0106)
            ├── Class: e
            └── stalk-root == c? (Entropy = 0.0053)
                ├── Class: e
                └── stalk-root == r? (Entropy = 0.0018)
                    ├── Class: e
                    └── spore-print-color == u? (Entropy = 0.0009)
                        ├── Class: e
                        └── Class: p


In [26]:
substitution_tree_mushroom.explain_notebook(sample_mushroom, sample_label_mushroom)